## Импорты и загрузка данных

In [3]:
import numpy as np
import pandas as pd

import os

In [49]:
path = os.getcwd()

fish_df = pd.read_csv(path + r"\data\db1\ref\fish.csv", error_bad_lines=False, sep=';')
prod_designate_df = pd.read_csv(path + r"\data\db1\ref\prod_designate.csv", error_bad_lines=False, sep=';')
prod_type_df = pd.read_csv(path + r"\data\db1\ref\prod_type.csv", error_bad_lines=False, sep=';')
regime_df = pd.read_csv(path + r"\data\db1\ref\regime.csv", error_bad_lines=False, sep=';')
region_df = pd.read_csv(path + r"\data\db1\ref\region.csv", error_bad_lines=False, sep=';')


catch_df = pd.read_csv(path + r'\data\db1\catch.csv', error_bad_lines=False, sep=',')

# Подмена данных + нормализация
def spoofing(id):
    return fish_df[fish_df['id_fish'] == id]['fish'].values[0]
catch_df['fish'] = catch_df['id_fish'].apply(spoofing)
catch_df['catch_volume'] = catch_df['catch_volume']*1000

product_df = pd.read_csv(path + r'\data\db1\product.csv', error_bad_lines=False, sep=',')

ext1 = pd.read_csv(path + r"\data\db2\Ext.csv", error_bad_lines=False, sep=',')
ext2 = pd.read_csv(path + r"\data\db2\Ext2.csv", error_bad_lines=False, sep=',')

In [51]:
# Сводная таблица по всем поступлениям согласно IdFish и выбранного промежутка времени.
def calculatingSummOfFish(fish_id, date_start, date_end):
    # смотрим по всем выловам этой рыбы
    data = catch_df[catch_df['id_fish'] == fish_id]
    # уникальные номера кораблей ловящих только эту рыбу
    ships = list(set(data['id_ves'].to_list()))
    
    # создаю датафрейм с владельцами и выловом по судам    
    total_df = pd.DataFrame()
    
    total_ships = []
    total_owners = []
    total_catches = []
    
    # добавляем в общую кашу дату и документируем владельцев
    for ship in ships:
        ship_data = data[data['id_ves'] == ship]
        owners = list(set(ship_data['id_own'].to_list()))
        
        for owner in owners:
            ship_owner_data = ship_data[ship_data['id_own'] == owner]
            
            try:
                ship_owner_end_data = ship_owner_data[ship_owner_data['date'] < date_end]
                ship_owner_start_end_data = ship_owner_end_data[ship_owner_end_data['date'] > date_start]
                total_catch = ship_owner_start_end_data['catch_volume'].sum()
            except e:
                print(e)
            
            if total_catch > 0:
                total_catches.append(total_catch)
                total_ships.append(ship)
                total_owners.append(owner)
            
    total_df['total_catches'] = total_catches
    total_df['ship'] = total_ships
    total_df['owner'] = total_owners
    
    return total_df

### Поиск аномалий

#### Осуществим с помощью вычленения ID рыбы и даты вылова (для простоты возьмем весь период, но можно брать любой промежуток)

In [44]:
fish_id = input('Введите ID рыбы\n')

default_data = (catch_df['date'][0], catch_df['date'][len(catch_df) - 1])
start, end = default_data

#### Возьмем уникальный лист владельцев судов

In [62]:
catcher_owners = catch_df['id_own'].unique()
traider_owners = ext1['id_own'].unique()

market_owners = []
for owner in traider_owners:
    if owner in catcher_owners:
        market_owners.append(owner)  

In [61]:
print(len(catcher_owners), len(traider_owners), len(market_owners))

350 159 121


#### Как видим, db2 не полностью покрывает всех владельцев, следовательно, будем использовать для поиска только тех, которые задействованы в обоих БД

In [45]:
def getAllOwnersIncomings(id, start, end):
    

'2022-01-01'

In [50]:
catch_df

,id_ves,date,id_region,id_fish,catch_volume,id_regime,permit,id_own,fish
0,4524386,2022-01-01,272,400,31874.00,41,852296,7115834,минтай
1,474886,2022-01-01,277,886,476.00,1,5401178,1809581,краб-стригун красный
2,3160994,2022-01-01,116,409,26310.00,15,3869431,5258490,путассу (северная)
3,1234400,2022-01-01,272,292,9541.00,1,2169057,1809581,треска
4,1234400,2022-01-01,272,113,62.00,1,2169057,1809581,палтус черный
...,...,...,...,...,...,...,...,...,...
85082,4992471,2022-04-20,108,410,1166.00,18,1518104,8379463,сайда
85083,7645033,2022-04-20,265,292,1345.00,97,1750243,1002633,треска
85084,5951077,2022-04-20,507,113,4.00,97,5099847,440996,палтус черный
85085,5292474,2022-04-20,206,147,505.05,97,229919,7829965,окунь золотистый
